In [1]:
import networkx as nx
import numpy as np
from scipy.optimize import linprog

In [2]:
%run "attack_graph_MARA.ipynb"

In [3]:
# Print nodes before
print("Nodes before:", list(attack_graph.nodes()))

Nodes before: [1, 2, 3, 4, 5, 6, 7, 8, 9]


In [4]:
def find_and_add_entry_node(graph):
    """
    Find nodes with no incoming edges and add virtual entry node if needed
    """
    # Find nodes with in-degree = 0 (no incoming edges)
    # Before: graph has nodes [1,2,3,4,5]
    # roots = [1] (node with no incoming edges)
    roots = [n for n,d in graph.in_degree() if d == 0]
    
    # If multiple root nodes found
    if len(roots) > 1:
        # Add new virtual entry node
        # Before: nodes = [1,2,3,4,5]
        # After: nodes = ["attacker_entry_node",1,2,3,4,5] 
        entry = "attacker_entry_node"
        graph.add_node(entry)
        
        # Add edges from entry node to all roots with weight 1
        # Creates edges like: ("attacker_entry_node" -> 1), ("attacker_entry_node" -> 2)
        for root in roots:
            graph.add_edge(entry, root, weight=1)
            
        return entry, graph
    else:
        # If single root, use it as entry
        return roots[0], graph

In [5]:
# Run function
entry, attack_graph = find_and_add_entry_node(attack_graph)

# Print nodes after 
print("Nodes after:", list(attack_graph.nodes()))

Nodes after: [1, 2, 3, 4, 5, 6, 7, 8, 9]


In [30]:
def process_target_nodes(graph):
    """
    Merge all target nodes (nodes with out-degree 0) into single target
    """
    # Find nodes with out-degree = 0 (target nodes)
    # Before: graph has nodes [1,2,3,4,5,6] where 5,6 are targets
    # target_list = [5,6]
    target_list = [n for n,d in graph.out_degree() if d == 0]
    
    # Create mapping for node contraction
    # Before: nodes = [1,2,3,4,5,6]
    # vertex_no = {1:1, 2:2, 3:3, 4:4, 5:8, 6:8} where 8 is joint vertex number
    joint_vertex = graph.number_of_nodes() - len(target_list) + 1
    vertex_no = {node: i+1 for i, node in enumerate(graph.nodes()) 
                if node not in target_list}
    
    # Add joint vertex number for all targets
    for target in target_list:
        vertex_no[target] = joint_vertex
        
    # Contract nodes according to mapping
    # This merges nodes with same mapping number
    graph = nx.quotient_graph(graph, 
                            lambda x, y: vertex_no[x] == vertex_no[y],
                            relabel=True)
    
    return graph, vertex_no

In [31]:
def generate_game_elements(graph, entry_node):
    """
    Generate attack routes and action spaces for both players
    """
    # Find new target after contraction (node with out-degree 0)
    # target_list = [8] (merged target node)
    target_list = [n for n,d in graph.out_degree() if d == 0]
    
    # Get all simple paths from entry to target
    # routes = [[1,2,8], [1,3,4,8], [1,5,6,7,8]]
    routes = list(nx.all_simple_paths(graph, entry_node, target_list[0]))
    
    # Get unique nodes across all paths
    # V = [1,2,3,4,5,6,7,8]
    V = sorted(list(set([node for path in routes for node in path])))
    
    # Get defender's action space (exclude entry, roots, targets)
    # as1 = [3,4,5,6,7]
    excluded = {entry_node, *target_list}
    if entry_node == "attacker_entry_node":
        excluded.update(set([path[1] for path in routes]))
    as1 = [n for n in V if n not in excluded]
    
    # Attacker's action space is the routes themselves
    as2 = routes
    
    return routes, V, as1, as2, target_list

In [32]:
def setup_game_parameters(V, routes, entry_node, target_list):
    """
    Setup initial probability distributions and game parameters
    """
    # Get list of possible adversary starting locations
    # V = [1,2,3,4,8], entry = 1, target = 8
    # adv_list = [2,3,4]
    adv_list = [n for n in V if n not in [entry_node, *target_list]]
    
    # Create uniform probability distribution for adversary locations
    # theta = {2: 0.33, 3: 0.33, 4: 0.33}
    theta = {loc: 1/len(adv_list) for loc in adv_list}
    
    # Get number of strategies for each player
    # m = number of attack paths, n = number of defender check locations
    m = len(routes)
    n = len(adv_list)
    
    return adv_list, theta, m, n

In [33]:
def calculate_location_distribution(path, check_node, avatar, theta, random_steps_fn, 
                                 attack_rate, defense_rate):
    """
    Calculate probability distribution of attacker location for one path
    """
    # Initialize distribution vector
    # L = {1:0, 2:0, 3:0, 4:0, 8:0}
    L = {node: 0 for node in path}
    
    if avatar in path:
        # Get route from avatar's position
        # path = [1,2,4,8], avatar = 2
        # route = [2,4,8]
        start_idx = path.index(avatar)
        route = path[start_idx:]
        
        # Get step distribution
        # pdf_d = [0.15, 0.30, 0.30, 0.20, 0.05]
        pdf_d = random_steps_fn(route, attack_rate, defense_rate)
        
        # Find cut point (where defender checks)
        # check_node = 4, route = [2,4,8]
        # cut_point = 2 (index of 4 in route)
        try:
            cut_point = min(route.index(check_node) + 1, len(route))
        except ValueError:
            cut_point = len(route)
            
        # Calculate payoff distribution
        if sum(pdf_d[:cut_point]) == 0:
            # If no probability mass before cut, put all at cut point
            payoff_distr = [0] * cut_point
            payoff_distr[-1] = 1
        else:
            # Normalize probabilities up to cut point
            payoff_distr = pdf_d[:cut_point] / sum(pdf_d[:cut_point])
            
        # Assign probabilities to locations
        for i, node in enumerate(route[:cut_point]):
            L[node] = payoff_distr[i]
            
    else:
        # If avatar not on path, stays at starting position
        L[avatar] = 1
        
    return L

In [34]:
def create_payoff_matrix(as1, as2, V, adv_list, theta, random_steps_fn, 
                        attack_rate, defense_rate, node_order):
    """Create payoff matrix for the game"""
    payoff_matrix = []
    print(f"Creating matrix with:")
    print(f"as1: {as1}")
    print(f"as2: {as2}")
    print(f"V: {V}")
    print(f"adv_list: {adv_list}")
    
    # ... rest of the function
    print(f"Final payoff matrix shape: {np.array(payoff_matrix).shape}")
    return np.array(payoff_matrix)

In [35]:
def solve_game(payoff_matrix, n, m, as1):
    """
    Create game object and compute optimal strategy
    
    Args:
        payoff_matrix: Matrix of payoffs for each defender-attacker strategy combination
        n: Number of defender strategies (check locations)
        m: Number of attacker strategies (paths)
        as1: List of defender check locations
        
    Returns:
        Dictionary containing optimal defense strategy and attacker success probability
    """
    # Reshape payoff matrix to (n x m) format
    # Before: Single list of distributions
    # After: Matrix where rows=defender actions, cols=attacker paths
    payoff_matrix = payoff_matrix.reshape(n, m, -1)
    
    # Create game object (using scipy.optimize.linprog)
    # This solves the zero-sum game using linear programming
    from scipy.optimize import linprog
    
    # Setup the linear programming problem
    c = np.zeros(n)
    A_ub = -payoff_matrix.reshape(m, n)
    b_ub = -np.ones(m)
    A_eq = np.ones((1, n))
    b_eq = [1]
    bounds = [(0, None) for _ in range(n)]
    
    # Solve for defender's optimal strategy
    result = linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=bounds)
    
    # Extract solution
    optimal_defense = dict(zip(as1, result.x))
    value = -1/result.fun if result.fun != 0 else float('inf')
    
    return {
        'optimal_defense': optimal_defense,
        'value': value
    }

In [36]:
def run_game(graph, attack_rate_list, defense_rate_list, random_steps_fn):
    """Main function to run the complete game analysis"""
    # Process entry nodes
    entry_node, graph = find_and_add_entry_node(graph)
    print(f"Entry node: {entry_node}")
    
    # Process target nodes
    graph, vertex_no = process_target_nodes(graph)
    
    # Generate game elements
    routes, V, as1, as2, target_list = generate_game_elements(graph, entry_node)
    print(f"Routes: {routes}")
    print(f"V: {V}")
    print(f"as1 (defender actions): {as1}")
    print(f"as2 (attacker paths): {as2}")
    print(f"target_list: {target_list}")
    
    # Setup initial parameters
    adv_list, theta, m, n = setup_game_parameters(V, routes, entry_node, target_list)
    print(f"adv_list: {adv_list}")
    print(f"m: {m}, n: {n}")
    
    # ... rest of the function

In [37]:
# If defense_rate_list not defined, use default
if 'defense_rate_list' not in locals():
    defense_rate_list = [0]
    
# If attack_rate_list not defined, use default  
if 'attack_rate_list' not in locals():
    attack_rate_list = [0]

In [38]:
# Execute game
results = run_game(attack_graph, attack_rate_list, defense_rate_list, random_steps)

# Display results
with open('experiment_1.log', 'r') as f:
    print(f.read())

NameError: name 'attack_graph' is not defined